In [177]:
import sys
import scipy.io
import numpy
import math
import csv
from operator import add

from gurobipy import *

# set the directory path
import os
folder_name = os.getcwd()

epsilon = 10e-6

import time
import copy

inf = 1e10

In [178]:
# # set of physical layer blocks (PRBs)
with open('B.csv', 'rb') as f:
    B_csv = csv.reader(f)
    B = list(B_csv)
    B = [item for sublist in B for item in sublist] # flatten list
    B = map(int, map(float, B)) # convert to int
    
# # set of latency services    
with open('Kl.csv', 'rb') as f:
    Kl_csv = csv.reader(f)
    Kl = list(Kl_csv)
    Kl = [item for sublist in Kl for item in sublist] # flatten list
    Kl = map(int, map(float, Kl)) # convert to int
    
# # set of capacity services    
with open('Kc.csv', 'rb') as f:
    Kc_csv = csv.reader(f)
    Kc = list(Kc_csv)
    Kc = [item for sublist in Kc for item in sublist] # flatten list
    Kc = map(int, map(float, Kc)) # convert to int
    
# # set of all services
K = Kl + Kc

# # set of resource units (RUs)
with open('I.csv', 'rb') as f:
    I_csv = csv.reader(f)
    I = list(I_csv)
    I = [item for sublist in I for item in sublist] # flatten list
    I = map(int, map(float, I)) # convert to int

In [179]:
# # matrix r
with open('r.csv', 'rb') as f:
    r_csv = csv.reader(f)
    r = list(r_csv)
    r = [ map(int,map(float,x)) for x in r] # convert to int

# # vector q, only for Kl
with open('q.csv', 'rb') as f:
    q_csv = csv.reader(f)
    q = list(q_csv)
    q = [item for sublist in q for item in sublist] # flatten list
    q = map(int, map(float, q)) # convert to int
    
# # matrix a
with open('a.csv', 'rb') as f:
    a_csv = csv.reader(f)
    a = list(a_csv)
    a = [ map(int,map(float,x)) for x in a] # convert to int

In [180]:
# if conflict_PRBs[b1][b2] == True, then the PRBs b1 b2 cannot be used simultaneously
with open('conflict_PRB.csv', 'rb') as f:
    conflict_PRB_csv = csv.reader(f)
    conflict_PRB = list(conflict_PRB_csv)
    conflict_PRB = [ map(lambda y: int(float(y))==1, x) for x in conflict_PRB]

In [181]:
with open('nB.csv', 'rb') as f:
    nB_csv = csv.reader(f)
    nB = list(nB_csv)
    nB = [item for sublist in nB for item in sublist] # flatten list
    nB = map(int, map(float, nB)) # convert to int

In [182]:
def getPRBShape(PRB_index):
    if PRB_index < nB[0]:
        PRB_shape = 0
    elif PRB_index < nB[0]+nB[1]:
        PRB_shape = 1
    elif PRB_index < nB[0]+nB[1]+nB[2]:
        PRB_shape = 2
    else:
        PRB_shape = 3
    return PRB_shape

In [183]:
utility = [ [ 0 for b in B ] for k in K ]
block_size = [4,4,4,4]
for k in K:
    for b in B:
        shape = getPRBShape(b)
        utility[k][b] = r[b][k]

In [184]:
def getAlpha(lam):
    alpha=[]
    for b in B:
        alpha_b = sum( lam[i]*a[b][i] for i in I ) 
        alpha.append(alpha_b)
    return alpha

In [185]:
# The argument lam is lambda
# The function returns matrix x, with only columns in Kc being computed.
# The columns in Kl of the returned matrix x are zero vectors.
def solveP2( lam ):
    PRB_alloc = [ -1 for b in B ] # PRB_alloc[b] is the index of service that PRB b should be allocated to
                                  # PRB_alloc[b]=-1 means that PRB b is not allocated
        
    sol_x = [ [ 0 for k in K ] for b in B ] # variables to be returned
    
    alpha = getAlpha(lam)
    for b in B:
        tmp_list = [ r[b][k]-alpha[b] for k in Kc ]
        if max(tmp_list) > 0:   # PRB is allocated only if r[b][k]-alpha[b] is positive
            PRB_alloc[b] = len(Kl) + numpy.argmax(tmp_list) # Kl is added such that the value of PRB_alloc[b] is 
                                                       # coherent with the corresponding indexed position in K
    
    # Convert PRB_alloc to matrix x
    for b in B:
        k = PRB_alloc[b] # indexed service
        if k >= 0: # indicating that PRB_alloc[b] != -1
            sol_x[b][k] = 1
    
    return sol_x

In [186]:
# The argument lam is lambda
# The function returns matrix x, with only columns in Kl being computed.
# The columns in Kc of the returned matrix x are zero vectors.
def solveP3( lam ):
    # create optimization model
    modelKl = Model('Integer Programming - Kl')
    modelKl.modelSense = GRB.MINIMIZE
    modelKl.setParam('OutputFlag', False) # slience output
    
    # create varialbes for modelKl:
    xKl = []
    for b in B:
        xKl_b = []
        for k in Kl:
            xKl_b.append(modelKl.addVar(vtype=GRB.BINARY))
        xKl.append(xKl_b)
    modelKl.update()
    
    # add constraints 
    for k in Kl:
        modelKl.addConstr( sum(r[b][k]*xKl[b][k] for b in B) >= q[k] )
    modelKl.update()

    # set objective function
    alpha = getAlpha(lam)
    modelKl.setObjective(
        sum( alpha[b]*xKl[b][k] for k in Kl for b in B )
    )
    
    # solve modelKl
    modelKl.optimize()
    
    # construct variables to be returned 
    sol_x = [ [ 0 for k in K ] for b in B ]
    for b in B:
        for k in Kl:
            sol_x[b][k] = int(xKl[b][k].x)
            
    return sol_x

In [187]:
def assignBlock(utility, x):
    sol_x = copy.deepcopy(x)
    collision = [ False for b in B ]
    util_Kl = numpy.array(utility[:len(Kl) :])
    util_Kc = numpy.array(utility[len(Kl): :])
    
    for b in B:
        if sum(sol_x[b]) > 0:
            for p in B: # set all PRB overlapping with b to be in collision
                if conflict_PRB[b][p] == True:
                    collision[p] = True

    to_be_alloc = copy.deepcopy(Kl)
    while len(to_be_alloc)>0 and collision.count(False)>0:
        [service, block] = numpy.unravel_index(numpy.argmax(util_Kl),util_Kl.shape)
#         print (service,block), util_Kl[service][block]
        service_bit = sum(r[b][service]*sol_x[b][service] for b in B)
        if service_bit < q[service] and not collision[block]:
            sol_x[block][service] = 1
            for p in B: # set all PRB overlapping with b to be in collision
                if conflict_PRB[block][p] == True:
                    collision[p] = True
        if service_bit >= q[service] and service in to_be_alloc:
            to_be_alloc.remove(service)
            for b in B:
                util_Kl[service][b] = -inf
        if collision[block]==True:
            for k in Kl:
                util_Kl[k][block] = -inf

    while collision.count(False)>0:
        [service, block] = numpy.unravel_index(numpy.argmax(util_Kc),util_Kc.shape)
        service += len(Kl)
        if not collision[block]:
            sol_x[block][service] = 1
            for p in B: # set all PRB overlapping with b to be in collision
                if conflict_PRB[block][p] == True:
                    collision[p] = True
        if collision[block] == True:
            for k in Kc:
                util_Kc[k-len(Kl)][block] = -inf

    return sol_x

In [188]:
def LPBased(rho):
    model = Model('Linear Programming')
    model.modelSense = GRB.MAXIMIZE
    model.setParam('OutputFlag', False) # slience output

    x = []
    for b in B:
        x_b = []
        for k in K:
            x_b.append(model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1))
        x.append(x_b)

    model.update()

    for k in Kl:
        model.addConstr( sum(r[b][k]*x[b][k] for b in B) >= q[k] )
    model.update()

    for i in I:
        model.addConstr( sum(a[b][i]*x[b][k] for k in K for b in B ) <= 1 )
    model.update()

    model.setObjective(
        sum( r[b][k]*x[b][k] for k in Kc for b in B )
    )

    model.optimize()
    
    x_count = [ [ x[b][k].x for b in B] for k in K ]
    for b in B:
        for k in K:
            if x[b][k].x < rho:
                x_count[k][b] = 0
    return x_count

In [189]:
def isFeasible(x):
    if (numpy.dot( numpy.dot(numpy.matrix(a).transpose(), 
                             numpy.matrix(x)), numpy.ones(len(K)) ) 
        > numpy.ones(len(I))).tolist()[0].count(True)>0:
        return False
    if (numpy.dot(numpy.multiply(numpy.matrix(r),numpy.matrix(x))[:,0:len(Kl)].transpose(),
                  numpy.ones(len(B)))<numpy.matrix(q)).tolist()[0].count(True)>0:
        return False
    return True

In [190]:
def filterX(x):
    filtered_x = copy.deepcopy(x)
    for k in Kl:
        service_bit = sum(r[b][k]*filtered_x[b][k] for b in B)
        rate_k = [ r[b][k]*filtered_x[b][k] for b in B ]
        while service_bit > q[k]:
#             print (k,b), rate_k[b]
            b = numpy.argmax(rate_k)
            if filtered_x[b][k] == 0 or rate_k[b] < 0: break
            if service_bit - rate_k[b]*filtered_x[b][k] >= q[k]:
                service_bit -= rate_k[b]*filtered_x[b][k]
                filtered_x[b][k] = 0
            rate_k[b] = -1
    return filtered_x

In [191]:
def getAssignment(x):
    solK = []
    for k in K:
        sol = [i for i,j in enumerate([int(x[b][k]) for b in B]) if j > 0]
        solK.append(sol)
    return solK

In [192]:
def LPLD(M, varphi, beta, gamma, mute=False):
    start_time = time.time()
    x_count_LP = LPBased(0)
    obj_LP =  sum(r[b][k]*x_count_LP[k][b] for k in Kc for b in B)
    
    lam = [1 for i in I]
    x_prev = [ [0 for k in K ] for b in B ]
    penalty = [ 1 for i in I ]
    x_count = [ [ 0 for b in B ] for k in K ]
    best_dual_sofar = 1e10
    eta = 0.95
    no_improve_count = 0

    x_fix_LP = []
    for rho in numpy.arange(0,0.95,0.025):
        xx_LP = [ [ 1 if x_count_LP[k][b]>=rho and k in Kl else 0 for k in K] for b in B ]
        xx_LP = filterX(xx_LP)
        x_fix_LP.append(xx_LP)
        
    best_obj_sofar = 0
    best_x_sofar = [ [ 0 for k in K ] for b in B ]
    for xx_LP in x_fix_LP:
        sol_x = assignBlock(x_count_LP, xx_LP)
        obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B)
        fixed_pairs = sum(xx_LP[b][k] for k in K for b in B)
        allocated_pairs = sum(sol_x[b][k] for k in K for b in B)
        if obj>best_obj_sofar and isFeasible(sol_x):
            best_obj_sofar = obj
            best_x_sofar = copy.deepcopy(sol_x)
        if not mute:
            print 'new solution - LP: ', 'fixed pairs=', fixed_pairs/(allocated_pairs*1.0),  'feasibility=', isFeasible(sol_x), 'obj = ', obj  
    if not mute:
        print 'best solution: ', 'feasibility=', isFeasible(best_x_sofar), ' obj=', best_obj_sofar          
    
    check_interval = 20
    
    for j in range(1, M+1): # k belongs to [1,100]
        
        xKc = solveP2(lam)
        xKl = solveP3(lam)
        x = (numpy.matrix(xKl) + numpy.matrix(xKc)).tolist() 

        # obtain the corresponding dual function value under current lambda

        dual = sum(r[b][k]*x[b][k] for k in Kc for b in B) + sum( lam[i]*(1-sum(a[b][i]*x[b][k] for k in K for b in B)) for i in I)
        if dual < best_dual_sofar + 0.005:
            no_improve_count = 0
            best_dual_sofar = dual
            gamma = min(gamma*1.05, 1.9)
        else:
            no_improve_count += 1

        if no_improve_count >=1:
            gamma = 0.95*gamma
            no_improve_count = 0

        numurator = abs(dual - obj_LP*beta)
        denominator = numpy.linalg.norm(penalty, 2)**2

        if j<varphi:
            mu = 10/float(j**0.5)
        else:
            mu = gamma*numurator/denominator

        penalty = (numpy.ones(len(I))- numpy.dot( numpy.dot(numpy.matrix(a).transpose(), numpy.matrix(x)), numpy.ones(len(K)))).tolist()[0]
        lam = map(lambda y:max(y,0), (numpy.matrix(lam) - mu*numpy.matrix(penalty)).tolist()[0])
        alpha = getAlpha(lam)

        x_count = (numpy.matrix(x_count) + numpy.matrix(x).transpose()).tolist()
        if not mute:
            print 'iteration', j, 'step length=', mu, 'dual=', best_dual_sofar, 'gamma=', gamma
        
        if j>0 and j%check_interval == 0:
            x_count_LD = [ [ x_count[k][b]/(j*1.0) for b in B] for k in K ]
            
            x_fix_LD = copy.deepcopy(x_fix_LP)
            for rho in numpy.arange(0,0.95,0.025):
                xx = [ [ 1 if x_count_LD[k][b]>=rho and k in Kl else 0 for k in K] for b in B ]
                xx = filterX(xx)
                x_fix_LD.append(xx)

            for xx in x_fix_LD:
                sol_x = assignBlock(x_count_LD, xx)
                obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B)
                fixed_pairs = sum(xx[b][k] for k in K for b in B)
                allocated_pairs = sum(sol_x[b][k] for k in K for b in B)
                if obj>best_obj_sofar and isFeasible(sol_x):
                    best_obj_sofar = obj
                    best_x_sofar = copy.deepcopy(sol_x)
                if not mute:
                    print 'new solution - LD: ', 'fixed pairs=', fixed_pairs/(allocated_pairs*1.0),  'feasibility=', isFeasible(sol_x), 'obj = ', obj  
            if not mute:
                print 'best solution: ', 'feasibility=', isFeasible(best_x_sofar), ' obj=', best_obj_sofar

    if not mute:       
        print 'LD upper bound = ', best_dual_sofar
        print 'Solving LD costs', time.time()-start_time, 'seconds'
    return best_obj_sofar, isFeasible(best_x_sofar)

In [193]:
def optSolver():
    model = Model('Interger Programming')
    model.modelSense = GRB.MAXIMIZE
    model.setParam('OutputFlag', False) # slience output
    model.setParam('TimeLimit', 3*60) # time_limit for gurobi
    
    sol_x = [ [ 0 for k in K ] for b in B ]

    x = []
    for b in B:
        x_b = []
        for k in K:
    #         x_b.append(model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1))
            x_b.append(model.addVar(vtype=GRB.BINARY))
        x.append(x_b)

    model.update()

    for k in Kl:
        model.addConstr( sum(r[b][k]*x[b][k] for b in B) >= q[k] )
    model.update()

    for i in I:
        model.addConstr( sum(a[b][i]*x[b][k] for k in K for b in B ) <= 1 )
    model.update()

    model.setObjective(
        sum( r[b][k]*x[b][k] for k in Kc for b in B )
    )

    model.optimize()
    
    if model.Status == GRB.OPTIMAL or model.Status == GRB.SUBOPTIMAL:
        for b in B:
            for k in K:
                sol_x[b][k] = x[b][k].x
            
    return sum( r[b][k]*sol_x[b][k] for k in Kc for b in B ), isFeasible(sol_x)

In [194]:
def Greedy():
    x_fix = [ [ 0 for k in K ] for b in B ]
    sol_x = assignBlock(utility,x_fix)
    fb = isFeasible(sol_x)
    obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B) if fb else 0
    return obj, fb

In [195]:
def LPRounding():
    x_count_LP = LPBased(0)
    x_fix = [ [ 0 for k in K ] for b in B ]
    sol_x = assignBlock(x_count_LP, x_fix)
    fb = isFeasible(sol_x)
    obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B) if fb else 0
    return obj, fb

In [196]:
def LPOnly():
    x_count_LP = LPBased(0)
    x_fix_LP = []
    for rho in numpy.arange(0,0.95,0.025):
        xx_LP = [ [ 1 if x_count_LP[k][b]>=rho and k in Kl else 0 for k in K] for b in B ]
        xx_LP = filterX(xx_LP)
        x_fix_LP.append(xx_LP)
        
    best_obj_sofar = 0
    best_x_sofar = [ [ 0 for k in K ] for b in B ]
    for xx_LP in x_fix_LP:
        sol_x = assignBlock(x_count_LP, xx_LP)
        obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B)
        fixed_pairs = sum(xx_LP[b][k] for k in K for b in B)
        allocated_pairs = sum(sol_x[b][k] for k in K for b in B)
        if obj>best_obj_sofar and isFeasible(sol_x):
            best_obj_sofar = obj
            best_x_sofar = copy.deepcopy(sol_x)
            
    return best_obj_sofar, isFeasible(best_x_sofar)

In [197]:
def LDOnly(M, varphi, beta, gamma, mute=False):
    start_time = time.time()
    x_count_LP = LPBased(0)
    obj_LP =  sum(r[b][k]*x_count_LP[k][b] for k in Kc for b in B)
    
    lam = [1 for i in I]
    x_prev = [ [0 for k in K ] for b in B ]
    penalty = [ 1 for i in I ]
    x_count = [ [ 0 for b in B ] for k in K ]
    best_dual_sofar = 1e10
    eta = 0.95
    no_improve_count = 0
        
    best_obj_sofar = 0
    best_x_sofar = [ [ 0 for k in K ] for b in B ]       
    
    check_interval = 20
    
    for j in range(1, M+1): # k belongs to [1,100]
        
        xKc = solveP2(lam)
        xKl = solveP3(lam)
        x = (numpy.matrix(xKl) + numpy.matrix(xKc)).tolist() 

        # obtain the corresponding dual function value under current lambda

        dual = sum(r[b][k]*x[b][k] for k in Kc for b in B) + sum( lam[i]*(1-sum(a[b][i]*x[b][k] for k in K for b in B)) for i in I)
        if dual < best_dual_sofar + 0.005:
            no_improve_count = 0
            best_dual_sofar = dual
            gamma = min(gamma*1.05, 1.9)
        else:
            no_improve_count += 1

        if no_improve_count >=1:
            gamma = 0.95*gamma
            no_improve_count = 0

        numurator = abs(dual - obj_LP*beta)
        denominator = numpy.linalg.norm(penalty, 2)**2

        if j<varphi:
            mu = 10/float(j**0.5)
        else:
            mu = gamma*numurator/denominator

        penalty = (numpy.ones(len(I))- numpy.dot( numpy.dot(numpy.matrix(a).transpose(), numpy.matrix(x)), numpy.ones(len(K)))).tolist()[0]
        lam = map(lambda y:max(y,0), (numpy.matrix(lam) - mu*numpy.matrix(penalty)).tolist()[0])
        alpha = getAlpha(lam)

        x_count = (numpy.matrix(x_count) + numpy.matrix(x).transpose()).tolist()
        if not mute:
            print 'iteration', j, 'step length=', mu, 'dual=', best_dual_sofar, 'gamma=', gamma
        
        if j>0 and j%check_interval == 0:
            x_count_LD = [ [ x_count[k][b]/(j*1.0) for b in B] for k in K ]
            
            x_fix_LD = []
            for rho in numpy.arange(0,0.95,0.025):
                xx = [ [ 1 if x_count_LD[k][b]>=rho and k in Kl else 0 for k in K] for b in B ]
                xx = filterX(xx)
                x_fix_LD.append(xx)

            for xx in x_fix_LD:
                sol_x = assignBlock(x_count_LD, xx)
                obj = sum(r[b][k]*sol_x[b][k] for k in Kc for b in B)
                fixed_pairs = sum(xx[b][k] for k in K for b in B)
                allocated_pairs = sum(sol_x[b][k] for k in K for b in B)
                if obj>best_obj_sofar and isFeasible(sol_x):
                    best_obj_sofar = obj
                    best_x_sofar = copy.deepcopy(sol_x)
                if not mute:
                    print 'new solution - LD: ', 'fixed pairs=', fixed_pairs/(allocated_pairs*1.0),  'feasibility=', isFeasible(sol_x), 'obj = ', obj  
            if not mute:
                print 'best solution: ', 'feasibility=', isFeasible(best_x_sofar), ' obj=', best_obj_sofar
    if not mute:      
        print 'LD upper bound = ', best_dual_sofar
        print 'Solving LD costs', time.time()-start_time, 'seconds'
    return best_obj_sofar, isFeasible(best_x_sofar)